# pandapower Optimal Power Flow
This is an introduction into the usage of the pandapower optimal power flow. It shows how to set the constraints and the cost factors into the pandapower element tables.

## Example Network

We use the following four bus example network for this tutorial:

<img src="pics/example_opf.png" width="50%">

We first create this network in pandapower:

In [1]:
import pandapower as pp
net = pp.create_empty_network()

#create buses
bus1 = pp.create_bus(net, vn_kv=220.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)

#create 220/110 kV transformer
pp.create_transformer(net, bus1, bus2, std_type="100 MVA 220/110 kV")

#create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=50., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus4, bus2, length_km=40., std_type='149-AL1/24-ST1A 110.0')

#create loads
pp.create_load(net, bus2, p_kw=60e3)
pp.create_load(net, bus3, p_kw=70e3)
pp.create_load(net, bus4, p_kw=10e3)

#create generators
pp.create_ext_grid(net, bus1, cost_per_kw=.1, vm_pu=1.00)
pp.create_gen(net, bus3, p_kw=-80*1e3, min_p_kw=0, max_p_kw=-80e3,vm_pu=1.01, controllable=True,
              cost_per_kw = .15)

pp.create_gen(net, bus4, p_kw=-100*1e3, min_p_kw=0, max_p_kw=-100e3, vm_pu=1.01, controllable=True,
              cost_per_kw = .12)

1

## Generator Dispatch Optimization

The classical application of an OPF is optimization of generator dispatch. We run an OPF:

In [2]:
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


let's check the results:

In [3]:
net.res_ext_grid

,p_kw,q_kvar
0,-144559.3764,-9192.99992


In [4]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-0.028436,-8601.814435,-16.426860,0.967619
1,-0.080673,-10594.697296,-13.481034,0.989756


We can see that all active power is provided by the external grid. This makes sense, because the external grid has the lowest cost of all generators and we did not define any constraints.

We define a simple function that calculates the summed costs:

In [5]:
def calc_costs(net):
    cost_gen = (-net.res_gen.p_kw * net.gen.cost_per_kw).sum()
    cost_eg = (-net.res_ext_grid.p_kw * net.ext_grid.cost_per_kw).sum()
    return (cost_gen + cost_eg) * 1e-3

And calculate the dispatch costs:

In [6]:
calc_costs(net)

14.45595158623664

### Transformer Constraint

Since all active power comes from the external grid and subsequently flows through the transformer, the transformer is overloaded with a loading of about 145%:

In [7]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,144559.376399,9192.999921,-143959.448402,15993.78371,599.927997,25186.783631,0.380136,0.759988,144.851388


We now limit the transformer loading to 50%:

In [8]:
net.trafo["max_loading_percent"] = 50

(the max_loading_percent parameter can also be specified directly when creating the transformer)
and run the OPF:

In [9]:
pp.runopp(net)

We can see that the transformer complies with the maximum loading:

In [10]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,49953.527334,-2147.249369,-49833.477544,5167.348248,120.049789,3020.098879,0.131215,0.262151,49.999656


And power generation is now split between the external grid and generator 1 (which is the second cheapest generation unit):

In [11]:
net.res_ext_grid

,p_kw,q_kvar
0,-49953.527352,2147.24937


In [12]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-0.280311,-2993.094181,-6.232780,0.985230
1,-93304.128507,-3453.267596,-1.237846,1.025709


This comes of course with an increase in dispatch costs:

In [13]:
calc_costs(net)

16.191890202723574

### Line Loading Constraints

Wen now look at the line loadings:

In [14]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_ka,loading_percent
0,19779.834885,-2479.406830,-19341.528367,1104.348550,438.306518,-1375.058280,0.104308,22.193125
1,-50658.191411,1888.745716,52783.580480,921.051915,2125.389070,2809.797631,0.270139,57.476425
2,30520.548121,2532.215803,-29946.357283,-2687.941435,574.190839,-155.725632,0.157324,33.473168


and run the OPF with a 50% loading constraint:

In [15]:
net.line["max_loading_percent"] = 50
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


Now the line loading constraint is complied with:

In [16]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_ka,loading_percent
0,16727.370958,-3203.510785,-16412.543224,1542.694314,314.827734,-1660.816471,0.088856,18.905569
1,-44451.003651,888.003392,46059.718758,811.619557,1608.715107,1699.622949,0.235000,49.999981
2,27532.897210,4042.154936,-27060.003950,-4420.233591,472.893260,-378.078655,0.143049,30.435913


And all generators are involved in supplying the loads:

In [17]:
net.res_ext_grid

,p_kw,q_kvar
0,-49787.580003,4603.457521


In [18]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-9136.450967,-2430.689358,-5.815435,0.993014
1,-83592.613615,-4853.769506,-1.511329,1.028887


This of course comes with a once again rising dispatch cost:

In [19]:
calc_costs(net)

16.380339279073926

### Voltage Constraints

Finally, we have a look at the bus voltage:

In [20]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,-49787.580003,4603.457521
1,1.006024,-3.408832,60000.000000,0.000000
2,0.993014,-5.815435,60863.549033,-2430.689358
3,1.028887,-1.511329,-73592.613615,-4853.769506


and constrain it:

In [21]:
net.bus["min_vm_pu"] = 1.0
net.bus["max_vm_pu"] = 1.02
pp.runopp(net)

We can see that all voltages are within the voltage band:

In [22]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,-49906.817982,3050.590592
1,1.004168,-3.421013,60000.000000,0.000000
2,1.000000,-5.976090,59278.074279,-14859.213746
3,1.020000,-1.366887,-71863.385929,9173.011326


And all generators are once again involved in supplying the loads:

In [23]:
net.res_ext_grid

,p_kw,q_kvar
0,-49906.817982,3050.590592


In [24]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-10721.925721,-14859.213746,-5.976090,1.00
1,-81863.385929,9173.011326,-1.366887,1.02


This of course comes once again with rising dispatch costs:

In [25]:
calc_costs(net)

16.42257696799004

## Minimizing Active Power Curtailment

Now we assume that the generators are renewable energy sources, and we want to feed in as much of the energy they can provide as possible without violating any constraints.

We assign negative costs to the generators and costs of zero for the external grid.

In [26]:
net.ext_grid.cost_per_kw = 0
net.gen.cost_per_kw = -1
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


Because of the negative costs, the OPF now maximizes power generation at the transformers:

In [27]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-46052.644816,-18912.998087,-0.485062,1.02
1,-85097.782114,19478.228436,2.869966,1.02


In [28]:
net.res_ext_grid

,p_kw,q_kvar
0,-11071.74798,153.958385


In [29]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.00,0.000000,-11071.747980,153.958385
1,1.00,-0.759444,60000.000000,0.000000
2,1.02,-0.485062,23947.355184,-18912.998087
3,1.02,2.869966,-75097.782114,19478.228436


In [30]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,11071.74798,-153.958385,-11013.579078,324.308603,58.168902,170.350217,0.029059,0.057831,11.072818


In [31]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_ka,loading_percent
0,-4940.672063,-7253.499354,5009.683533,5024.003563,69.011470,-2229.495791,0.046064,9.800775
1,-28957.038717,13888.994524,29770.861583,-13899.328668,813.822866,-10.334143,0.169066,35.971544
2,45326.920531,-5578.899768,-44045.748858,6929.190751,1281.171673,1350.290984,0.235000,49.999973


Obviously the voltage profile was the limiting factor for the generator feed-in. If we relax this constraint a little bit:

In [32]:
net.bus.max_vm_pu = 1.05
pp.runopp(net)

We see an increased feed-in of the generators:

In [33]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-79999.991970,-17252.186816,2.328271,1.050000
1,-66517.876366,10225.945538,3.665995,1.040019


In [34]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,4542.414746,8523.970418
1,1.010373,0.298333,60000.000000,0.000000
2,1.050000,2.328271,-9999.991970,-17252.186816
3,1.040019,3.665995,-56517.876366,10225.945538
